In [ ]:
import tensorflow as tf

In [2]:
g = tf.Graph()  # 创建一个Graph对象

在模型中有两个“全局”风格的Variable对象：`global_step` 和 `total_output` 。它们本质上是全局的，因此在声明它们时需要与数据流图的其他节点区分开，并把它们放入自己的名称作用域。`trainable= False`的设置使得该Variable对象对象不会对模型造成影响（因为没有任何训练的步骤），但该设置明确指定了这些Variable对象只能通过手工设置。

# 构建数据流图

In [3]:
with g.as_default():   # 将Graph对象设为默认对象
    with tf.name_scope('variables'):
        # 记录数据流图运行次数的 Variable 对象
        # 这是一种常见的范式，在整个API中，这种范式会频繁出现
        global_step = tf.Variable(0, dtype= tf.float32, trainable= False, name= 'global_step')
        
        # 追踪该模型的所有输出随时间的累加和的 Variable 对象
        total_output = tf.Variable(0.0, dtype= tf.float32, trainable= False, name= 'total_output')
        
    with tf.name_scope('transformation'):  # 该模型的核心变换部分
        
        # 独立的输入层
        with tf.name_scope('input'):
            # 创建输出占位符，用于接收任意长度的向量
            a = tf.placeholder(tf.float32, shape= [None], name= 'input_placeholder_a')
            
        # 独立的中间层
        with tf.name_scope('intermediate_layer'):
            # 对整个向量实施乘法和加法
            b = tf.reduce_prod(a, name= 'prod_b')
            c = tf.reduce_sum(a, name= 'sum_c')
            
        # 独立的输出层
        with tf.name_scope('output'):
            output = tf.add(b, c, name= 'output')
        
        # 对变量进行更新
        with tf.name_scope('update'):
            # 用最新的输出更新Variable对象total_output
            update_total = total_output.assign(output)
            # 将Variable对象global_step增 1 ，只要数据流图运行，该操作便需要进行
            increment_step = global_step.assign_add(1)
            
        with tf.name_scope('summaries'):
            avg = tf.div(update_total, tf.cast(increment_step, tf.float32), name= 'average')
            
            # 为输出节点创建汇总数据
            tf.summary.scalar('Output', output)
            tf.summary.scalar('Sum of outputs over time', update_total)
            tf.summary.scalar('Average of outputs over time', avg)
            
        with tf.name_scope('global_ops'):
            # 初始化Op
            init = tf.global_variables_initializer()
            # 将所有汇总数据合并到一个Op中
            merged_summaries = tf.summary.merge_all()

INFO:tensorflow:Summary name Sum of outputs over time is illegal; using Sum_of_outputs_over_time instead.
INFO:tensorflow:Summary name Average of outputs over time is illegal; using Average_of_outputs_over_time instead.


*为什么将 `tf.summary.merge_all()` $Op$ 放在“$global\_ops$”名称作用域，而非放在“$summaries$”作用域?*
- 一般而言，将`tf.summary.merge_all()` 与其他全局$Op$放在一起是最佳做法。我们的数据流图只为汇总数据设置了一个环节，但这并不妨碍去想象一个拥有`Variable`、`Op`和名称作用域等的不同汇总数据的数据流图。通过保持 `tf.summary.merge_all()` 的分离，可确保用户无需记忆放置它的特定“summary” 代码块，从而比较容易找到该$Op$。

# 运行数据流图

In [7]:
sess = tf.Session(graph= g)
# 保存汇总数据
writer = tf.summary.FileWriter('E:/Graphs/xin_graph', g)
sess.run(init)
def run_graph(input_tensor):
    '''
    运行计算图
    '''
    feed_dict = {a: input_tensor}
    _, step, summary = sess.run([output, increment_step, merged_summaries],feed_dict= feed_dict)

    writer.add_summary(summary, global_step= step)

In [8]:
# 用不同的输入运行该数据流图
run_graph([2, 8])
run_graph([3, 1, 3, 3])
run_graph([8])
run_graph([1, 2, 3])
run_graph([11, 4])
run_graph([4, 1])
run_graph([7, 3, 1])
run_graph([6, 3])
run_graph([0, 2])
run_graph([4, 5 ,6])

# 将汇总数据写入磁盘
writer.flush()
writer.close()
sess.close()